<a href="https://colab.research.google.com/github/wdavid93/OpenClassRoom/blob/main/test_spark_autre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# L'exécution de cette cellule démarre l'application Spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1702904777221_0001,pyspark,idle,Link,Link,None,
1,application_1702904777221_0002,pyspark,idle,Link,Link,None,
2,application_1702904777221_0003,pyspark,idle,Link,Link,None,
3,application_1702904777221_0004,pyspark,idle,Link,Link,None,✔


In [13]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Création de la SparkSession:**

In [14]:
#PATH = os.getcwd()

#PATH_Data = os.path.join(PATH, 'data', 'Test1')

PATH = 's3://wdp8bucket'
PATH_Data = PATH+'/Test1'
PATH_Result = os.path.join(PATH, 'data', 'Results')

# Vérifier et créer le répertoire 'data/Test1' s'il n'existe pas
if not os.path.exists(PATH_Data):
    os.makedirs(PATH_Data)

# Vérifier et créer le répertoire 'data/Results' s'il n'existe pas
if not os.path.exists(PATH_Result):
    os.makedirs(PATH_Result)

print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)
# PATH:        /mnt/c/Users/Home/Desktop/OpenClassrooms/Projet8/Projet
# PATH_Data:   /mnt/c/Users/Home/Desktop/OpenClassrooms/Projet8/Projet/data/Test1
# PATH_Result: /mnt/c/Users/Home/Desktop/OpenClassrooms/Projet8/Projet/data/Results

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://wdp8bucket
PATH_Data:   s3://wdp8bucket/Test1
PATH_Result: s3://wdp8bucket/data/Results

**Création de SparkContext**

In [15]:
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Affichage des infos de Spark en cours d'execution**

In [16]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On charge les images en format binaire**
- On indique qu'on ne veut charger uniquement que des fichiers jpg
- On lui indique aussi de charger les objets inclus dans les sous dossiers

## Extraction des features et export sous forme de parquets

In [17]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

On modifie l'objet "images" pour avoir un path + label

In [18]:
import pandas as pd
from PIL import Image
import numpy as np
import io

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.sql.functions import rand

images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
random_images = images.orderBy(rand()).limit(5)
print(random_images.printSchema())
print(random_images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------+-------------------+
|path                                                 |label              |
+-----------------------------------------------------+-------------------+
|s3://wdp8bucket/Test1/Beetroot/190_100.jpg           |Beetroot           |
|s3://wdp8bucket/Test1/Apple Golden 3/270_100.jpg     |Apple Golden 3     |
|s3://wdp8bucket/Test1/Apple Red 2/200_100.jpg        |Apple Red 2        |
|s3://wdp8bucket/Test1/Apple Red Delicious/120_100.jpg|Apple Red Delicious|
|s3://wdp8bucket/Test1/Apple Braeburn/170_100.jpg     |Apple Braeburn     |
+-----------------------------------------------------+-------------------+

None

**Chargement modèle MobileNetV2 (préentrainé avec imagenet)**
https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c

In [20]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 0s 0us/step

**On supprime la dernières couche pour que la "nouvelle" derniere couche soit un (1,1,1280)**

In [21]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On regarde a quoi il ressemble**

In [22]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

**On partage le modèle dans tout les noeuds du cluster:**
- Minimise la communication entre les noeuds et augmente les performances

In [23]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On cré une fonction model_fn():**
- Il charge le modèle préentrainé MobileNetV2
- Les poids sont non entrainables
- On supprime la dernière couche
- On définit les poids du nouveau modèle en utilisant les poids diffusés

In [24]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On cré une fonction preprocess(content):**
- Redimensionne les images 100x100 en images 224x224
- Convertit les images en tableaux numpy

In [25]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On cré une fonction featurize_series(model, content_series):**
- Prend en entrée un modèle et une série pandas d’images brutes (content_series)
- Applique la fonction preprocess()
- Utilise le modèle pour prédire les featires de chaque image dans le tableau numpy (preds)
- Aplati ces features en vecteurs pour faciliter leur stockage dans les DataFrames Spark (output)

In [26]:
def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On cré une fonction featurize_udf(content_series_iter):**
- **UDF = User-Defined function** (Utilise pandas pour manipuler les données dans Spark)
- **@pandas_udf("array-float"):** Cette UDF renvoie une sortie qui sera une liste de nombres à virgule flottante.
- Execute **model_fn()**
- Dans la boucle, on appelle la fonction **featurize_series()**

In [27]:
from pyspark.sql.pandas.functions import pandas_udf
from typing import Iterator, Tuple
import pandas as pd

@pandas_udf("array<float>")
def featurize_udf(content_series_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On va executer l'extraction de features sur nos images:**
- **images.repartition(20):** Cette ligne répartit le DataFrame images en 20 partitions
- Retourne un DataFrame avec les chemins des images, leurs labels, et leurs caractéristiques extraites

In [28]:
features_df = images.sample(False, 1.0).repartition(20).select(col("path"),
                                                               col("label"),
                                                               featurize_udf("content").alias("features")
                                                              )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://wdp8bucket/data/Results

**On enregistre les résultats de features_df dans 20 parquets**

In [30]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Import des parquets et execution de la PCA

In [33]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                 path  ...                                           features
0       s3://wdp8bucket/Test1/Cantaloupe 1/30_100.jpg  ...  [0.003774497, 0.0, 0.0, 0.0, 0.0, 0.0, 0.57025...
1      s3://wdp8bucket/Test1/Cantaloupe 1/240_100.jpg  ...  [0.0087437825, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0558...
2       s3://wdp8bucket/Test1/Apple Red 3/210_100.jpg  ...  [0.8126136, 0.07296174, 0.0, 0.0, 0.061739784,...
3          s3://wdp8bucket/Test1/Carambula/30_100.jpg  ...  [0.14472091, 0.0, 0.0, 0.0, 1.7372555, 0.00946...
4       s3://wdp8bucket/Test1/Apple Red 2/270_100.jpg  ...  [0.8811968, 0.006273552, 0.0, 0.0, 0.0, 0.0, 0...
..                                                ...  ...                                                ...
311  s3://wdp8bucket/Test1/Apple Golden 3/270_100.jpg  ...  [0.14147554, 0.60201305, 0.0, 0.0, 0.045595217...
312  s3://wdp8bucket/Test1/Apple Golden 2/200_100.jpg  ...  [0.1552659, 0.3921705, 0.028614784, 0.0, 0.140...
313       

In [35]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

On a bien 1280 features pour chaque image

In [36]:
# df = df.sample(frac=0.1, random_state=42) # sample si pas assez de memoire

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**On va déterminer le nb de composantes principales nécéssaires pour expliquer 95% de la variance**

In [37]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import PCA
from pyspark.sql.functions import udf

spark.sparkContext.setLogLevel("ERROR")

# Conversion du DataFrame Pandas en DataFrame PySpark
df['features'] = df['features'].apply(lambda x: Vectors.dense(x))
df_spark = spark.createDataFrame(df)

# Création d'un VectorAssembler qui combine une liste de colonnes en une seule colonne de vecteur.
vecAssembler = VectorAssembler(inputCols=["features"], outputCol="features_vec")

# Utilisation du VectorAssembler pour transformer notre DataFrame.
df_spark = vecAssembler.transform(df_spark)

variances = []
total_features = len(df_spark.select("features").first()[0])
step_size = int(total_features * 0.01) # 1% du nombre total de caractéristiques
# Initialisation de la liste des résultats
resultats = []

for k in range(1, total_features + 1, step_size):
    # Création d'une instance de PCA qui prend en entrée la colonne "features_vec" et renvoie la colonne "pcaFeatures".
    pca = PCA(k=k, inputCol="features_vec", outputCol="pcaFeatures")

    # Entraînement du modèle PCA.
    model = pca.fit(df_spark)

    # Ajout de la variance expliquée au tableau des variances.
    explained_variance = sum(model.explainedVariance)
    variances.append(explained_variance)

    # Ajout du nombre de caractéristiques et de la variance expliquée à la liste des résultats
    resultats.append((k, explained_variance))

    # Si la variance expliquée est égale à 1, arrêtez la boucle.
    if explained_variance >= 0.8:
        break

resultats.append(((total_features, 1.0)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
resultats

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, 0.14860503981553783), (13, 0.6681468546639453), (25, 0.790889549302143), (37, 0.8448392019188032), (1280, 1.0)]

**On cré un graphe qui va nous permettre de déterminer le nb de composantes principales optimales**

In [40]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [41]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Séparation des données en deux listes pour les axes x et y
x_data = [i[0] for i in resultats]
y_data = [i[1]*100 for i in resultats]

# Création du graphe
plt.figure(figsize=(10,6))
plt.plot(x_data, y_data, marker='o', label='Variance expliquée')

# Ajout des titres et labels
plt.title('Explication de la variance en fonction du Nb de composantes principales')
plt.xlabel('Nombre de composantes principales')
plt.ylabel('Explication de la variance (%)')

# Trouver la valeur x pour laquelle y atteint 80%
x_80 = np.interp(80, y_data, np.round(x_data))

# Tracer une ligne verticale à cette position x
plt.axvline(x=x_80, color='g', label='80% de variance expliquée')

# Ajouter la valeur de x en dessous du trait vertical
plt.text(x_80+30, 7, f'{math.ceil(x_80)}', color='g', ha='center')

# Afficher la légende
plt.legend()

# Affichage du graphe
plt.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'matplotlib'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'matplotlib'



In [38]:
Nb_composants = math.ceil(x_80)
Nb_composants

12

**On va effectuer une PCA sur un nombre k de tel sorte à ce qu'on ait 80% de la variance expliquée:**

In [43]:
Nb_composants = 12

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

spark.sparkContext.setLogLevel("ERROR")

# Conversion du DataFrame Pandas en DataFrame PySpark
df['features'] = df['features'].apply(lambda x: Vectors.dense(x))
df_spark = spark.createDataFrame(df)

# Création d'un VectorAssembler qui combine une liste de colonnes en une seule colonne de vecteur.
vecAssembler = VectorAssembler(inputCols=["features"], outputCol="features_vec")

# Utilisation du VectorAssembler pour transformer notre DataFrame.
df_spark = vecAssembler.transform(df_spark)

# Création d'un objet PCA
pca = PCA(k=Nb_composants, inputCol="features_vec", outputCol="pcaFeatures")

# Entraînement du modèle PCA sur les données
model = pca.fit(df_spark)

# Transformation des données avec le modèle PCA
result = model.transform(df_spark)

# Suppression de la colonne 'features_vec'
result = result.drop('features_vec')

# Affichage des résultats
result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------------+--------------------+
|                path|              label|            features|         pcaFeatures|
+--------------------+-------------------+--------------------+--------------------+
|s3://wdp8bucket/T...|       Cantaloupe 1|[0.00377449695952...|[1.13120953336349...|
|s3://wdp8bucket/T...|       Cantaloupe 1|[0.00874378252774...|[1.04384063096103...|
|s3://wdp8bucket/T...|        Apple Red 3|[0.81261360645294...|[-6.2512283482881...|
|s3://wdp8bucket/T...|          Carambula|[0.14472091197967...|[3.89896599540207...|
|s3://wdp8bucket/T...|        Apple Red 2|[0.88119679689407...|[-8.4848522178685...|
|s3://wdp8bucket/T...| Apple Crimson Snow|[0.06765466183423...|[-14.951749390863...|
|s3://wdp8bucket/T...| Apple Red Yellow 2|[0.94529974460601...|[-4.2883091277030...|
|s3://wdp8bucket/T...| Apple Crimson Snow|[0.10368233919143...|[-13.027254427283...|
|s3://wdp8bucket/T...|            Apricot|[1.13314878940582...|[-

**On cré un nouveau PATH qui sera PATH_Results_PCA (pour enregistrer les features PCA)**

In [51]:
#PATH = os.getcwd()
PATH = 's3://wdp8bucket'

PATH_Data = os.path.join(PATH, 'data', 'Test1')
PATH_Result_PCA = os.path.join(PATH, 'data', 'Results_PCA')

# Vérifier et créer le répertoire 'data/Results' s'il n'existe pas
if not os.path.exists(PATH_Result_PCA):
    os.makedirs(PATH_Result_PCA)
    
# Vérifier et créer le répertoire 'data/Results' s'il n'existe pas
if not os.path.exists(PATH_Result_PCA):
    os.makedirs(PATH_Result_PCA)
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Results_PCA: '+PATH_Result_PCA)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://wdp8bucket
PATH_Data:   s3://wdp8bucket/data/Test1
PATH_Results_PCA: s3://wdp8bucket/data/Results_PCA

In [52]:
%ls 

DS_P8_notebook2_emr_transform.ipynb     test_phil.ipynb
fruits.zip                              test_spark_autre.ipynb
jupyterhub_cookie_secret                test_spark_t12122023.ipynb
jupyterhub-proxy.pid                    Untitled1.ipynb
jupyterhub.sqlite                       Untitled2.ipynb
P8_01_notebook_sample_009.ipynb         Untitled.ipynb
P8_Notebook_Linux_EMR_PySpark_VP.ipynb


**On enregistre ces résultats dans le format parquet**

In [53]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, DenseVector
from pyspark.sql.types import ArrayType, FloatType

# UDF pour convertir Vector en list
vector_to_list = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

# Appliquer la conversion
result = result.withColumn("features", vector_to_list(result["features"]))
result = result.withColumn("pcaFeatures", vector_to_list(result["pcaFeatures"]))

# Écrire dans Parquet
result.write.mode('overwrite').parquet(PATH_Result_PCA)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df = pd.read_parquet(PATH_Result_PCA, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                 path  ...                                        pcaFeatures
0       s3://wdp8bucket/Test1/Cantaloupe 1/30_100.jpg  ...  [1.1312095, -1.4673606, -9.190808, -4.807373, ...
1      s3://wdp8bucket/Test1/Cantaloupe 1/240_100.jpg  ...  [1.0438406, -1.1090343, -9.611828, -4.4438276,...
2       s3://wdp8bucket/Test1/Apple Red 3/210_100.jpg  ...  [-6.2512283, -0.5664162, -5.4235625, -6.191817...
3          s3://wdp8bucket/Test1/Carambula/30_100.jpg  ...  [3.898966, -0.5759972, -0.23123911, -3.1906826...
4       s3://wdp8bucket/Test1/Apple Red 2/270_100.jpg  ...  [-8.484852, 0.6502148, -6.499008, -2.2582593, ...
..                                                ...  ...                                                ...
311  s3://wdp8bucket/Test1/Apple Golden 3/270_100.jpg  ...  [-0.664033, -1.470687, -12.027138, 3.8882143, ...
312  s3://wdp8bucket/Test1/Apple Golden 2/200_100.jpg  ...  [-0.09198229, -1.0596961, -9.27793, 5.232645, ...
313       

In [56]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [57]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(316, 4)